In [1]:
from ultralytics import YOLO
import cv2
import math
import requests
from datetime import date
from datetime import datetime
import json

In [ ]:

# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("C:/sameer/FYP_face_onyx/runs/detect/train/weights/best.pt")

# object classes
classNames = ["Hannan", "Arham", "Sameer"]


while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            # print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            isUnknown = True if confidence < 0.5 else False
            # print("Class name -->", 'Unknown' if confidence < 0.5 else classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, 'Unknown' if confidence < 0.5 else classNames[cls], org, font, fontScale, color, thickness)
           
            keys = {
                'Hannan' : 'wPnHYXndB6fQkNmX83uKA4E1iP02',
                'Arham' : 'vPXvYEjxf1Xb54HVHEHklVZUC9x2',
                'Sameer' : '7UKgVpNtT2UYeKeM2PMPB6LCPjF2'
                
                 }

            url = 'https://faceonyx-app-59085-default-rtdb.firebaseio.com/alerts.json'
            current_date = date.today().strftime("%Y-%m-%d")
            current_time = datetime.now().strftime("%H:%M:%S")
            custom_json = {
                keys[classNames[cls]]:[
                    {
                        "date" : current_date,
                        "time" : current_time,
                        "authorized": True,
                        "user": [classNames[cls]]
                    }
                ]
            }
            custom_json = json.dumps(custom_json)
            print(custom_json)
            res = requests.patch(url, data = custom_json)
            print(res)
            #if not isUnknown:
                # Code to trigger ignition event on response

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
from datetime import date
from datetime import datetime
keys = {
    'Sameer' : '7UKgVpNtT2UYeKeM2PMPB6LCPjF2',
    'Hannan' : 'wPnHYXndB6fQkNmX83uKA4E1iP02',
    'Arham' : 'vPXvYEjxf1Xb54HVHEHklVZUC9x2'
}



In [ ]:
from ultralytics import YOLO
import cv2
import math
import requests
from datetime import date
from datetime import datetime
import json
# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("C:/sameer/FYP_face_onyx/runs/detect/train/weights/best.pt")

# object classes
classNames = ["Hannan", "Arham", "Sameer","unknown"]


while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            isUnknown = True if confidence < 0.5 else False
            print("Class name -->", 'Unknown' if confidence < 0.5 else classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, 'Unknown' if confidence < 0.5 else classNames[cls], org, font, fontScale, color, thickness)
           
           
            #if not isUnknown:
                # Code to trigger ignition event on response

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load YOLO model
model = YOLO("C:/sameer/FYP_face_onyx/runs/detect/train/weights/best.pt")

# Object classes
classNames = ["Hannan", "Arham", "Sameer"]

def process_image(img_path):
    # Check if the file exists
    if not os.path.exists(img_path):
        print(f"Error: The file does not exist at the path: {img_path}")
        return

    # Check file permissions
    if not os.access(img_path, os.R_OK):
        print(f"Error: The file cannot be read (permissions issue): {img_path}")
        return

    # Read the image
    img = cv2.imread(img_path)

    # Check if the image was successfully loaded
    if img is None:
        print(f"Error: Unable to read the image from the path: {img_path}")
        return

    # Resize the image if needed
    max_width, max_height = 640, 640
    h, w = img.shape[:2]
    if h > max_height or w > max_width:
        scaling_factor = min(max_width/w, max_height/h)
        img = cv2.resize(img, (int(w*scaling_factor), int(h*scaling_factor)))

    # Convert image to RGB if needed (YOLO models typically expect RGB input)
    if img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    elif img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Run YOLO model
    results = model(img)

    # Draw bounding boxes and labels on the image
    for result in results:
        boxes = result.boxes
        if boxes:
            for box in boxes:
                # Extract bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                
                # Draw bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)

                # Confidence
                confidence = box.conf[0]

                # Class name
                cls = int(box.cls[0])
                class_name = 'Unknown' if confidence < 0.5 else classNames[cls]

                # Draw class label
                org = (x1, y1 - 10)  # Position text above the bounding box
                cv2.putText(img, class_name, org, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Display the image with bounding boxes and labels
    cv2.imshow('Image', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert RGB back to BGR for display
    cv2.waitKey(0)  # Wait for a key press to close the image window
    cv2.destroyAllWindows()

def main():
    # List of image paths
    img_paths = [  # Example uploaded image path
        "C:/sameer/FYP_face_onyx/dataset/images/train/IMG_6127.JPEG"
    ]

    for img_path in img_paths:
        process_image(img_path)

if __name__ == "__main__":
    main()


In [ ]:
from ultralytics import YOLO
import cv2
import os
import requests
import json
from datetime import date, datetime

# Load YOLO model
model = YOLO("C:/sameer/FYP_face_onyx/runs/detect/train/weights/best.pt")

# Object classes
classNames = ["Hannan", "Arham", "Sameer"]

# Firebase keys
keys = {
    'Hannan': 'wPnHYXndB6fQkNmX83uKA4E1iP02',
    'Arham': 'vPXvYEjxf1Xb54HVHEHklVZUC9x2',
    'Sameer': '7UKgVpNtT2UYeKeM2PMPB6LCPjF2'
}

# Firebase URL
url = 'https://faceonyx-app-59085-default-rtdb.firebaseio.com/alerts.json'

def process_image(img_path):
    # Check if the file exists
    if not os.path.exists(img_path):
        print(f"Error: The file does not exist at the path: {img_path}")
        return

    # Check file permissions
    if not os.access(img_path, os.R_OK):
        print(f"Error: The file cannot be read (permissions issue): {img_path}")
        return

    # Read the image
    img = cv2.imread(img_path)

    # Check if the image was successfully loaded
    if img is None:
        print(f"Error: Unable to read the image from the path: {img_path}")
        return

    # Resize the image if needed
    max_width, max_height = 640, 640
    h, w = img.shape[:2]
    if h > max_height or w > max_width:
        scaling_factor = min(max_width/w, max_height/h)
        img = cv2.resize(img, (int(w*scaling_factor), int(h*scaling_factor)))

    # Convert image to RGB if needed (YOLO models typically expect RGB input)
    if img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
    elif img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Run YOLO model
    results = model(img)

    # Draw bounding boxes and labels on the image
    for result in results:
        boxes = result.boxes
        if boxes:
            for box in boxes:
                # Extract bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                
                # Draw bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)

                # Confidence
                confidence = box.conf[0]

                # Class name
                cls = int(box.cls[0])
                class_name = 'Unknown' if confidence < 0.5 else classNames[cls]

                # Draw class label
                org = (x1, y1 - 10)  # Position text above the bounding box
                cv2.putText(img, class_name, org, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # If the person is identified, send data to Firebase
                if class_name != 'Unknown':
                    current_date = date.today().strftime("%Y-%m-%d")
                    current_time = datetime.now().strftime("%H:%M:%S")
                    custom_json = {
                        keys[class_name]: [
                            {
                                "date": current_date,
                                "time": current_time,
                                "authorized": True,
                                "user": [class_name]
                            }
                        ]
                    }
                    custom_json = json.dumps(custom_json)
                    print(custom_json)
                    res = requests.patch(url, data=custom_json)
                    print(res)

    # Display the image with bounding boxes and labels
    cv2.imshow('Image', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert RGB back to BGR for display
    cv2.waitKey(0)  # Wait for a key press to close the image window
    cv2.destroyAllWindows()

def main():
    # List of image paths
    img_paths = [
        "C:/sameer/FYP_face_onyx/dataset/images/train/IMG_6725.JPEG",
        "C:/sameer/FYP_face_onyx/dataset/images/train/IMG_6536.JPEG",  # Example uploaded image path
        "C:/sameer/FYP_face_onyx/dataset/images/train/IMG_6186.JPEG"
    ]

    for img_path in img_paths:
        process_image(img_path)

if __name__ == "__main__":
    main()
